# Classification of genomic sequences

+ heavily inspired by https://www.tensorflow.org/tutorials/keras/text_classification

In [ ]:
!pip install biopython

In [42]:
import urllib.request
from pathlib import Path
from Bio import SeqIO
import numpy as np
import gzip
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

## Reshaping data from FASTA to txt

In [3]:
classes = ['notpresent', 'present']
sets = ['train', 'valid']

for c in classes:
    for s in sets:
        urllib.request.urlretrieve(f"https://github.com/simecek/dspracticum2020/raw/master/lecture_08/assignment/g4/g4_{c}_{s}.fa.gz", f"g4_{c}_{s}.fa.gz")

for c in classes:
    for s in sets:
        Path(f"data/{s}/{c}").mkdir(parents=True, exist_ok=True)

for c in classes:
    for s in sets:
        with gzip.open(f"g4_{c}_{s}.fa.gz", "rt") as handle:
            for record in SeqIO.parse(handle, "fasta"):
                id = record.id
                with open(f"data/{s}/{c}/{id}.txt", "w") as fw:
                    fw.writelines([" ".join(str(record.seq))])


## Reading input data

In [4]:
batch_size = 128

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/train/',
    batch_size=batch_size,
    class_names=classes)

Found 210000 files belonging to 2 classes.


In [7]:
# Look at some example sequences
for seq_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Sequence", seq_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

Sequence b'N N G G C A T A A G A G T G T G T G T G T G T G T G T G T G T G T G T G T G T A T A C A C A C A A A C T G T A T A T A T G T G C T T G T G T G T A A T T G A A A T A C A T A T G T C A C A T A T C A T A C G T A C T A T G G C A T A T T A G G G T G T A A T A T T T A T A T T T G T G T A T A T A A T A A T A T T G A T G T T A A T G A A G T A T C C A T C A G T T T C T A T T T T C T T T A G T T T G T N N N'
Label 0
Sequence b'N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N T A T G G C T C A C T G C A G C C T G G A A C T C G G G C T C A A G C G A T C C T C C C A C C T C A G C C T C C T G A G T A G C T G G G A C T A C A G G T G T G T G C C A C C A A A T C C A G C T A A T T T T T G N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N N'
Label 0
Sequence b'A T G T A G A T G A C A A A A T C T G A G A G T T A T T T A T T T A A T A A A G A G A G G T T T G T A T G T G T G T G G G G G T G T G G G T G T G G G

In [9]:
# See labels
print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])

Label 0 corresponds to notpresent
Label 1 corresponds to present


In [11]:
raw_valid_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'data/valid/',
    batch_size=batch_size,
    class_names=classes)

Found 90000 files belonging to 2 classes.


In [32]:
vectorize_layer = TextVectorization(output_mode='int')

train_text = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(train_text)
# vectorize_layer.set_vocabulary(vocab=np.asarray(['a', 'c', 't', 'g', 'n'])) -> why do I not want to use this?

In [33]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label
  # originally with one-hot encoding: return vectorize_layer(text)-2, label -> why it is different in embedding?

train_ds = raw_train_ds.map(vectorize_text)
valid_ds = raw_valid_ds.map(vectorize_text)

In [34]:
# retrieve a batch (of 32 sequences and labels) from the dataset
seq_batch, label_batch = next(iter(raw_train_ds))
first_seq, first_label = seq_batch[0], label_batch[0]
print("Seq", first_seq)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized seq", vectorize_text(first_seq, first_label))

Seq tf.Tensor(b'N N N G G G C A G G G T A T C T G T G T C C G G T T G G G T C G G A A G C G A G G A A G G A C G G A C G G A C A G A G A G G G A C C G G G G A G A C G G G T G G G G A T G G G G C G A C G T C C T G T G G G G G T T C T A C T T T C C C G T T C C T C T T C C T C C C G G A C C C C C T G G A C C G A T G G T C C T G A A G G C C T G T T T G T C A A C A G T C C G T G A G C C G T A C G G G G C T C C A C N N', shape=(), dtype=string)
Label present
Vectorized seq (<tf.Tensor: shape=(1, 200), dtype=int64, numpy=
array([[6, 6, 6, 3, 3, 3, 5, 2, 3, 3, 3, 4, 2, 4, 5, 4, 3, 4, 3, 4, 5, 5,
        3, 3, 4, 4, 3, 3, 3, 4, 5, 3, 3, 2, 2, 3, 5, 3, 2, 3, 3, 2, 2, 3,
        3, 2, 5, 3, 3, 2, 5, 3, 3, 2, 5, 2, 3, 2, 3, 2, 3, 3, 3, 2, 5, 5,
        3, 3, 3, 3, 2, 3, 2, 5, 3, 3, 3, 4, 3, 3, 3, 3, 2, 4, 3, 3, 3, 3,
        5, 3, 2, 5, 3, 4, 5, 5, 4, 3, 4, 3, 3, 3, 3, 3, 4, 4, 5, 4, 2, 5,
        4, 4, 4, 5, 5, 5, 3, 4, 4, 5, 5, 4, 5, 4, 4, 5, 5, 4, 5, 5, 5, 3,
        3, 2, 5, 5, 5, 5, 5, 4, 3, 3

In [35]:
# lookup the token (string) that each integer corresponds to 
print("0 ---> ",vectorize_layer.get_vocabulary()[0]) # why there is no 0? Look below to see what I mean, all numbers have assigned nukleotide but not 0
print(" 1 ---> ",vectorize_layer.get_vocabulary()[1])
print(" 2 ---> ",vectorize_layer.get_vocabulary()[2])
print(" 3 ---> ",vectorize_layer.get_vocabulary()[3])
print(" 4 ---> ",vectorize_layer.get_vocabulary()[4])
print(" 5 ---> ",vectorize_layer.get_vocabulary()[5])
print(" 6 ---> ",vectorize_layer.get_vocabulary()[6])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

0 --->  
 1 --->  [UNK]
 2 --->  a
 3 --->  g
 4 --->  t
 5 --->  c
 6 --->  n
Vocabulary size: 7


## Model training

In [11]:
# one-hot encoding - NOT USED
onehot_layer = keras.layers.Lambda(lambda x: tf.one_hot(tf.cast(x,'int64'), 4))

model_lstm = tf.keras.Sequential([
    onehot_layer,
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32, return_sequences=False),
    keras.layers.Dense(1, activation="sigmoid")]) 

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  

In [38]:
# embedding
embedding_dim = 16
max_features = 7 # is 7 ok? I set it to the vocabulary size as in the text classification example mentioned at the beggining

model_lstm = tf.keras.Sequential([
    keras.layers.Embedding(max_features + 1, embedding_dim), # why +1 ?
    keras.layers.Dropout(0.2),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32, return_sequences=False),
    keras.layers.Dense(1, activation="sigmoid")]) 

model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  

In [40]:
epochs = 10 
history = model_lstm.fit(
    train_ds,
    epochs=epochs,
    validation_data = valid_ds)

Epoch 1/10
1641/1641 [==============================] - 123s 75ms/step - loss: 0.3205 - accuracy: 0.8784 - val_loss: 0.2532 - val_accuracy: 0.9162
Epoch 2/10
1641/1641 [==============================] - 127s 77ms/step - loss: 0.2295 - accuracy: 0.9209 - val_loss: 0.1743 - val_accuracy: 0.9366
Epoch 3/10
1641/1641 [==============================] - 122s 74ms/step - loss: 0.1561 - accuracy: 0.9425 - val_loss: 0.1352 - val_accuracy: 0.9485
Epoch 4/10
1641/1641 [==============================] - 126s 77ms/step - loss: 0.1346 - accuracy: 0.9486 - val_loss: 0.1202 - val_accuracy: 0.9548
Epoch 5/10
1641/1641 [==============================] - 128s 78ms/step - loss: 0.1205 - accuracy: 0.9546 - val_loss: 0.1183 - val_accuracy: 0.9566
Epoch 6/10
1641/1641 [==============================] - 128s 78ms/step - loss: 0.1112 - accuracy: 0.9584 - val_loss: 0.1043 - val_accuracy: 0.9621
Epoch 7/10
1641/1641 [==============================] - 128s 78ms/step - loss: 0.1009 - accuracy: 0.9626 - val_loss: 0